<a href="https://colab.research.google.com/github/visiont3lab/deep-learning-course/blob/main/colab/ExcerciseSineApprox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import plotly.graph_objects as go
from torch import nn
from torch.utils.data import DataLoader
from torch import optim
import torch
from torch import nn
from torchsummary import summary
#!pip install torchsummary
import torch.nn.functional as F
from torch.utils.data import TensorDataset,Dataset
from torchvision import datasets
from torchvision import transforms
from sklearn.model_selection import train_test_split
import numpy as np

# Loss function pytorch: https://neptune.ai/blog/pytorch-loss-functions

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(1,60)
        self.fc2 = nn.Linear(60,40)
        self.fc3 = nn.Linear(40,1)
    def forward(self,x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

class CustomTensorDataset(Dataset):

    def __init__(self, x,y):
        self.x = x
        self.y = y

    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        return x, y

    def __len__(self):
        return self.x.shape[0]


def metrics_batch(target, output):
    mse = torch.sum((output - target) ** 2)
    return mse

def loss_batch(loss_func, xb,yb,yb_h, opt=None):
    # obtain loss
    loss = loss_func(yb_h, yb)
    # obtain permormance metric 
    metric_b = metrics_batch(yb,yb_h)
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
    return loss.item(), metric_b

def loss_epoch(model, loss_func, dataset_dl, opt,device):
    loss = 0.0
    metric = 0.0
    len_data = len(dataset_dl.dataset)

    # Get batch data
    for xb,yb in dataset_dl:    
        # Send to cuda the data (batch size)
        xb = xb.type(torch.float32).to(device)
        yb = yb.to(device)

        # obtain model output 
        yb_h = model(xb)

        # Loss and Metric Calculation
        loss_b, metric_b = loss_batch(loss_func, xb,yb,yb_h,opt)
        loss += loss_b
        if metric_b is not None:
            metric+=metric_b 
    
    loss /=len_data
    metric /=len_data
    return loss, metric

def train_val(epochs, model, loss_func, opt, train_dl,val_dl,device):
    for epoch in range(epochs):
        model.train()
        train_loss,train_metric = loss_epoch(model, loss_func, train_dl, opt,device)
        model.eval()
        with torch.no_grad():
            val_loss, val_metric = loss_epoch(model, loss_func, val_dl,opt=None,device=device)
        accuracy = 100*val_metric
        print("epoch: %d, train_loss: %.6f, val loss: %.6f, accuracy: %.2f" % (epoch,train_loss, val_loss,accuracy))

# Setup GPU Device
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:0")

# Load and Preprocess data 
x = 1*np.linspace(-2,3,6000)
y = np.sin(5*x)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.33, random_state=4)
x_train = torch.from_numpy(x_train).unsqueeze(1).type(torch.float32)
y_train = torch.from_numpy(y_train).unsqueeze(1).type(torch.float32)
x_val = torch.from_numpy(x_val).unsqueeze(1).type(torch.float32)
y_val = torch.from_numpy(y_val).unsqueeze(1).type(torch.float32)

# Transformation
train_ds = CustomTensorDataset(x_train, y_train)
val_ds = CustomTensorDataset(x_val, y_val)

# Create Data loader
train_dl = DataLoader(train_ds, batch_size=50)
val_dl = DataLoader(val_ds, batch_size=50)

# Define Model, Loss , Optimizer
model = Net()
model.to(device)
#print(model)
# By default model is hosted on CPU
print("Model Parameter Device: ", next(model.parameters()).device)
summary(model, input_size=tuple(x_train.shape))
loss_func = nn.MSELoss(reduction="sum") 
opt = optim.Adam(model.parameters(), lr=1e-2)

# Train
num_epochs = 100
train_val(num_epochs,model, loss_func,opt, train_dl, val_dl,device)

# Save Models (It save last weights)
path2weigths="./weights.pt"
torch.save(model.state_dict(),path2weigths)

Model Parameter Device:  cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1             [-1, 4020, 60]             120
            Linear-2             [-1, 4020, 40]           2,440
            Linear-3              [-1, 4020, 1]              41
Total params: 2,601
Trainable params: 2,601
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 3.10
Params size (MB): 0.01
Estimated Total Size (MB): 3.12
----------------------------------------------------------------
epoch: 0, train_loss: 0.502121, val loss: 0.500059, accuracy: 50.01
epoch: 1, train_loss: 0.457991, val loss: 0.445913, accuracy: 44.59
epoch: 2, train_loss: 0.378062, val loss: 0.377292, accuracy: 37.73
epoch: 3, train_loss: 0.292343, val loss: 0.295579, accuracy: 29.56
epoch: 4, train_loss: 0.213137, val loss: 0.175530, accuracy: 17.

In [3]:
# Data 
x = 1*np.linspace(-5,6,6000)
y = np.sin(5*x)

# Model Trained Load
md = Net()
weights = torch.load(path2weigths)
md.load_state_dict(weights)
device = torch.device("cpu")
md = md.to(device)
#print(next(md.parameters()))

y_hat_vec = md(torch.from_numpy(x).unsqueeze(1).type(torch.float32))
y_hat_vec = y_hat_vec.detach().numpy()
y_hat_vec = y_hat_vec.reshape(-1)


# Deploy
fig = go.Figure()
fig.add_traces( go.Scatter(x=x, y=y, name="Real"))
fig.add_traces( go.Scatter(x=x, y=y_hat_vec, name="Estimate"))
fig.show()
